In [ ]:
from transformers import DistilBertTokenizer,DistilBertConfig, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, f1_score
from sklearn.metrics import roc_curve, auc
import torch
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader
from transformers import TrainerCallback
from torch.utils.data import DataLoader

#torch.cuda.set_per_process_memory_fraction(device=0, fraction=1.0)

In [ ]:
config_rubert = DistilBertConfig.from_pretrained("DeepPavlov/distilrubert-tiny-cased-conversational")
model_state_rubert = torch.load("/kaggle/input/rubert-pre-trained/pytorch/v1/1/model_distilrubert_latest.pth")

# Создание экземпляра модели
model_trained_rubert = DistilBertForSequenceClassification(config_rubert)

# Установка состояния модели из загруженного файла
model_trained_rubert.load_state_dict(model_state_rubert)
tokenizer_rubert = DistilBertTokenizer.from_pretrained("DeepPavlov/distilrubert-tiny-cased-conversational")

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification,RobertaConfig

config_roberta = RobertaConfig.from_pretrained("blinoff/roberta-base-russian-v0")
model_state_roberta = torch.load("/kaggle/input/roberta-pre-traind/pytorch/v1/1/model_roberta_latest.pth")

# Создание экземпляра модели
model_trained_roberta = RobertaForSequenceClassification(config_roberta)

# Установка состояния модели из загруженного файла
model_trained_roberta.load_state_dict(model_state_roberta)
tokenizer_roberta = RobertaTokenizer.from_pretrained("blinoff/roberta-base-russian-v0")

In [ ]:
from transformers import DebertaTokenizer, DebertaForSequenceClassification, DebertaConfig

config_deberta = DebertaConfig.from_pretrained("deepvk/deberta-v1-distill")
model_state_deberta = torch.load("/kaggle/input/deberta-pre-trained/pytorch/v1/1/model_deberta_v2.pth")

# Создание экземпляра модели
model_trained_deberta = DebertaForSequenceClassification(config_deberta)

# Установка состояния модели из загруженного файла
model_trained_deberta.load_state_dict(model_state_deberta)
tokenizer_deberta = DebertaTokenizer.from_pretrained("deepvk/deberta-v1-distill")

In [ ]:
df = pd.read_csv("/kaggle/input/cooked-shorter-small/cooked_dataset_cutted_small.csv")  
texts = df["clean_text"].tolist()
labels = df["generated"].tolist()


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

    
encodings_rubert = tokenizer_rubert(texts, truncation=True, padding=True, max_length=512)
encodings_roberta = tokenizer_roberta(texts, truncation=True, padding=True)
encodings_deberta = tokenizer_deberta(texts, truncation=True, padding=True)


dataset_rubert = CustomDataset(encodings_rubert, labels)
dataset_roberta = CustomDataset(encodings_roberta, labels)
dataset_deberta = CustomDataset(encodings_deberta, labels)

In [ ]:
batch_size = 8
dataloader = DataLoader(dataset_roberta, batch_size=batch_size)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_trained_roberta = model_trained_roberta.to(device)
model_trained_roberta.eval()


true_labels = []
predicted_labels = []
logits = []
with torch.no_grad():
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)  # Предположим, что input_ids находится на нулевой позиции в батче
        attention_mask = batch['attention_mask'].to(device)  # Предположим, что attention_mask находится на первой позиции в батче
        outputs = model_trained_roberta(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        predicted_labels.extend(predictions.detach().cpu().numpy())
        true_labels.extend(batch['labels'])
        logits.extend(outputs.logits)

print(len(true_labels), ' ', len(predicted_labels))


assert len(true_labels) == len(predicted_labels), "Количество истинных меток не совпадает с количеством предсказанных меток"

if len(true_labels) == len(predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    auc_score = roc_auc_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    print("Accuracy:", accuracy)
    print("Recall:", recall)
    print("ROC AUC:", auc_score)
    print("F1 score:", f1)
else:
    print("Количество истинных меток не совпадает с количеством предсказанных меток.")


In [ ]:
batch_size = 8
dataloader = DataLoader(dataset_rubert, batch_size=batch_size)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_trained_rubert = model_trained_rubert.to(device)
model_trained_rubert.eval()


true_labels_rubert = []
predicted_labels_rubert = []
logits_rubert = []
with torch.no_grad():
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)  # Предположим, что input_ids находится на нулевой позиции в батче
        attention_mask = batch['attention_mask'].to(device)  # Предположим, что attention_mask находится на первой позиции в батче
        outputs = model_trained_rubert(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        predicted_labels_rubert.extend(predictions.detach().cpu().numpy())
        true_labels_rubert.extend(batch['labels'])
        logits_rubert.extend(outputs.logits)

print(len(true_labels_rubert), ' ', len(predicted_labels_rubert))


assert len(true_labels_rubert) == len(predicted_labels_rubert), "Количество истинных меток не совпадает с количеством предсказанных меток"

if len(true_labels_rubert) == len(predicted_labels_rubert):
    accuracy = accuracy_score(true_labels_rubert, predicted_labels_rubert)
    recall = recall_score(true_labels_rubert, predicted_labels_rubert)
    auc_score = roc_auc_score(true_labels_rubert, predicted_labels_rubert)
    f1 = f1_score(true_labels_rubert, predicted_labels_rubert)
    print("Accuracy:", accuracy)
    print("Recall:", recall)
    print("ROC AUC:", auc_score)
    print("F1 score:", f1)
else:
    print("Количество истинных меток не совпадает с количеством предсказанных меток.")


In [ ]:
batch_size = 8
dataloader = DataLoader(dataset_deberta, batch_size=batch_size)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_trained_deberta = model_trained_deberta.to(device)
model_trained_deberta.eval()


true_labels_deberta = []
predicted_labels_deberta = []
logits_deberta = []
with torch.no_grad():
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)  # Предположим, что input_ids находится на нулевой позиции в батче
        attention_mask = batch['attention_mask'].to(device)  # Предположим, что attention_mask находится на первой позиции в батче
        outputs = model_trained_deberta(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        predicted_labels_deberta.extend(predictions.detach().cpu().numpy())
        true_labels_deberta.extend(batch['labels'])
        logits_deberta.extend(outputs.logits)

print(len(true_labels_deberta), ' ', len(predicted_labels_deberta))


assert len(true_labels_deberta) == len(predicted_labels_deberta), "Количество истинных меток не совпадает с количеством предсказанных меток"

if len(true_labels_deberta) == len(predicted_labels_deberta):
    accuracy = accuracy_score(true_labels_deberta, predicted_labels_deberta)
    recall = recall_score(true_labels_deberta, predicted_labels_deberta)
    auc_score = roc_auc_score(true_labels_deberta, predicted_labels_deberta)
    f1 = f1_score(true_labels_deberta, predicted_labels_deberta)
    print("Accuracy:", accuracy)
    print("Recall:", recall)
    print("ROC AUC:", auc_score)
    print("F1 score:", f1)
else:
    print("Количество истинных меток не совпадает с количеством предсказанных меток.")


In [ ]:
df_test = pd.read_csv("/kaggle/input/test-cooked/cooked_dataset_test_small.csv")  

texts_test = df_test["clean_text"].tolist()
labels_test = df_test["generated"].tolist()


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

encodings_rubert_test = tokenizer_rubert(texts_test, truncation=True, padding=True, max_length=512)
encodings_roberta_test = tokenizer_roberta(texts_test, truncation=True, padding=True)
encodings_deberta_test = tokenizer_deberta(texts_test, truncation=True, padding=True)



dataset_rubert_test = CustomDataset(encodings_rubert_test, labels_test)
dataset_roberta_test = CustomDataset(encodings_roberta_test, labels_test)
dataset_deberta_test = CustomDataset(encodings_deberta_test, labels_test)

In [ ]:
from torch.utils.data import DataLoader
model_trained_roberta
batch_size = 8
dataloader_test = DataLoader(dataset_roberta_test, batch_size=batch_size)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_trained_roberta = model_trained_roberta.to(device)
model_trained_roberta.eval()


true_labels_test = []
predicted_labels_test = []
logits_test = []
with torch.no_grad():
    for batch in dataloader_test:
        input_ids = batch['input_ids'].to(device)  # Предположим, что input_ids находится на нулевой позиции в батче
        attention_mask = batch['attention_mask'].to(device)  # Предположим, что attention_mask находится на первой позиции в батче
        outputs = model_trained_roberta(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        predicted_labels_test.extend(predictions.detach().cpu().numpy())
        true_labels_test.extend(batch['labels'])
        logits_test.extend(outputs.logits)

print(len(true_labels_test), ' ', len(predicted_labels_test))


assert len(true_labels_test) == len(predicted_labels_test), "Количество истинных меток не совпадает с количеством предсказанных меток"

if len(true_labels_test) == len(predicted_labels_test):
    accuracy = accuracy_score(true_labels_test, predicted_labels_test)
    recall = recall_score(true_labels_test, predicted_labels_test)
    auc_score = roc_auc_score(true_labels_test, predicted_labels_test)
    f1 = f1_score(true_labels_test, predicted_labels_test)
    print("Accuracy:", accuracy)
    print("Recall:", recall)
    print("ROC AUC:", auc_score)
    print("F1 score:", f1)
else:
    print("Количество истинных меток не совпадает с количеством предсказанных меток.")


In [ ]:
batch_size = 8
dataloader_test = DataLoader(dataset_rubert_test, batch_size=batch_size)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_trained_rubert = model_trained_rubert.to(device)
model_trained_rubert.eval()


true_labels_rubert_test = []
predicted_labels_rubert_test = []
logits_rubert_test = []
with torch.no_grad():
    for batch in dataloader_test:
        input_ids = batch['input_ids'].to(device)  # Предположим, что input_ids находится на нулевой позиции в батче
        attention_mask = batch['attention_mask'].to(device)  # Предположим, что attention_mask находится на первой позиции в батче
        outputs = model_trained_rubert(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        predicted_labels_rubert_test.extend(predictions.detach().cpu().numpy())
        true_labels_rubert_test.extend(batch['labels'])
        logits_rubert_test.extend(outputs.logits)

print(len(true_labels_rubert_test), ' ', len(predicted_labels_rubert_test))


assert len(true_labels_rubert_test) == len(predicted_labels_rubert_test), "Количество истинных меток не совпадает с количеством предсказанных меток"

if len(true_labels_rubert_test) == len(predicted_labels_rubert_test):
    accuracy = accuracy_score(true_labels_rubert_test, predicted_labels_rubert_test)
    recall = recall_score(true_labels_rubert_test, predicted_labels_rubert_test)
    auc_score = roc_auc_score(true_labels_rubert_test, predicted_labels_rubert_test)
    f1 = f1_score(true_labels_rubert_test, predicted_labels_rubert_test)
    print("Accuracy:", accuracy)
    print("Recall:", recall)
    print("ROC AUC:", auc_score)
    print("F1 score:", f1)
else:
    print("Количество истинных меток не совпадает с количеством предсказанных меток.")


In [ ]:
batch_size = 8
dataloader_test = DataLoader(dataset_deberta_test, batch_size=batch_size)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_trained_deberta = model_trained_deberta.to(device)
model_trained_deberta.eval()


true_labels_deberta_test = []
predicted_labels_deberta_test = []
logits_deberta_test = []
with torch.no_grad():
    for batch in dataloader_test:
        input_ids = batch['input_ids'].to(device)  # Предположим, что input_ids находится на нулевой позиции в батче
        attention_mask = batch['attention_mask'].to(device)  # Предположим, что attention_mask находится на первой позиции в батче
        outputs = model_trained_deberta(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        predicted_labels_deberta_test.extend(predictions.detach().cpu().numpy())
        true_labels_deberta_test.extend(batch['labels'])
        logits_deberta_test.extend(outputs.logits)

print(len(true_labels_deberta_test), ' ', len(predicted_labels_deberta_test))


assert len(true_labels_deberta) == len(predicted_labels_deberta), "Количество истинных меток не совпадает с количеством предсказанных меток"

if len(true_labels_deberta_test) == len(predicted_labels_deberta_test):
    accuracy = accuracy_score(true_labels_deberta_test, predicted_labels_deberta_test)
    recall = recall_score(true_labels_deberta_test, predicted_labels_deberta_test)
    auc_score = roc_auc_score(true_labels_deberta_test, predicted_labels_deberta_test)
    f1 = f1_score(true_labels_deberta_test, predicted_labels_deberta_test)
    print("Accuracy:", accuracy)
    print("Recall:", recall)
    print("ROC AUC:", auc_score)
    print("F1 score:", f1)
else:
    print("Количество истинных меток не совпадает с количеством предсказанных меток.")
